In [ ]:
# Install necessary Python libraries
!python3 -m pip install --upgrade langchain deeplake openai gitpython tiktoken python-magic

In [ ]:
# Import necessary libraries
import os
import shutil
from getpass import getpass
from git import Repo

# Set up API keys
os.environ['OPENAI_API_KEY'] = getpass("OpenAi Token:")
os.environ['ACTIVELOOP_TOKEN'] = getpass('Activeloop Token:')

In [ ]:
# Function to clone repository
def clone_repository(github_user, reponame, branch='main'):
    repo_url = f'https://github.com/{github_user}/{reponame}.git'
    local_path = os.path.join('.', reponame)
    
    if os.path.exists(local_path):
        shutil.rmtree(local_path)
    
    repo = Repo.clone_from(repo_url, local_path, branch=branch)
    return local_path

In [ ]:
# Clone the repository
github_user = 'lguibr'
reponame = 'pongo'
branch = 'main' # Optional, defaults to 'main'
local_path = clone_repository(github_user, reponame, branch)

In [ ]:
# Load the documents
import os
from langchain.document_loaders import TextLoader
import magic

root_dir = local_path

ignored_paths = [
    '.git',
    '/.venv/',
    # Add more paths to ighere
]


def is_text_file(filepath):
    file_type = magic.from_file(filepath)
    return 'text' in file_type

docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    print(f"Directory: {dirpath}")
    if any(ignored_path in dirpath for ignored_path in ignored_paths):
        print(f"Ignoring the directory: {dirpath}")
        continue  # Skip this directory if it contains an ignored path

    for file in filenames:
        filepath = os.path.join(dirpath, file)
        if is_text_file(filepath):
            print(f"Loading file: {file}")
            try:
                loader = TextLoader(filepath, encoding='utf-8')
                
                content = loader.load_and_split()
                print(f'filepath:\n{filepath}\n')
                docs.extend(content)
            except Exception as e:
                print(f"Error loading file: {file}, error: {e}")
                pass

print(f"Loaded {len(docs)} documents")


In [ ]:
# Split documents
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=4000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

# After splitting documents
print(f"Created {len(texts)} texts")


In [ ]:
# Steup DEEPLAKE_ACCOUNT_NAME

DEEPLAKE_ACCOUNT_NAME="lgpelin92"
DATASET_NAME="pongo"

In [ ]:
# Create embeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
db = DeepLake.from_documents(texts, embeddings, dataset_path=f"hub://{DEEPLAKE_ACCOUNT_NAME}/{DATASET_NAME}")


In [ ]:
# Set up LangChain and DeepLake
db = DeepLake(dataset_path=f"hub://{DEEPLAKE_ACCOUNT_NAME}/{DATASET_NAME}", read_only=True, embedding_function=embeddings)

model='gpt-4'
timeout=6000

from langchain.chains.llm import LLMChain
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT, QA_PROMPT
from langchain.chains.question_answering import load_qa_chain


model = ChatOpenAI(
    model=model,
    timeout=timeout,
    streaming=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,   
)


qa = ConversationalRetrievalChain.from_llm(model, retriever=db.as_retriever())

In [ ]:
from IPython.display import display, Markdown, FileLink, clear_output

chat_history = []

while True:
    question = input("Enter your question (type 'exit' to quit): ")
    
    if question.lower() == 'exit':
        break
    clear_output()
    display(Markdown(f"-> **Question**: {question}"))
    display(Markdown(f"**Answer**:"))
    result = qa({"question": question, "chat_history": chat_history})
    answer = result['answer']
    clear_output()
    display(Markdown(f"-> **Question**: {question}"))
    display(Markdown(f"**Answer**: {answer}\n"))
    
    chat_history.append((question, answer))

clear_output()    
display(Markdown("\nChat history:"))

for q, a in chat_history:
    display(Markdown(f"-> **Question**: {q}"))
    display(Markdown(f"**Answer**: {a}\n"))
        
## Save chat_history to a text file
filename = "chat_history.txt"
with open(filename, "w") as f:
    for q, a in chat_history:
        f.write(f"-> **Question**: {q}\n")
        f.write(f"**Answer**: {a}\n\n")

# Offer user to download the file
FileLink(filename, result_html_prefix="Click here to download chat history: ")
